In [1]:
import os

captcha_images_folder = "captcha_images"
captchas = [
    os.path.join(captcha_images_folder, f) for f in os.listdir(captcha_images_folder)
]

In [2]:
print(captchas)

['captcha_images\\222A.png', 'captcha_images\\223B.png', 'captcha_images\\226E.png', 'captcha_images\\226S.png', 'captcha_images\\229C.png', 'captcha_images\\22AA.png', 'captcha_images\\22B4.png', 'captcha_images\\22C9.png', 'captcha_images\\22CA.png', 'captcha_images\\22DQ.png', 'captcha_images\\22F6.png', 'captcha_images\\22H2.png', 'captcha_images\\22KD.png', 'captcha_images\\22N9.png', 'captcha_images\\22NM.png', 'captcha_images\\22PH.png', 'captcha_images\\22PT.png', 'captcha_images\\22PZ.png', 'captcha_images\\22RG.png', 'captcha_images\\22RH.png', 'captcha_images\\22SM.png', 'captcha_images\\22SW.png', 'captcha_images\\22TB.png', 'captcha_images\\22UM.png', 'captcha_images\\22V5.png', 'captcha_images\\22W4.png', 'captcha_images\\22W7.png', 'captcha_images\\22W8.png', 'captcha_images\\22XB.png', 'captcha_images\\22XC.png', 'captcha_images\\22XJ.png', 'captcha_images\\22YD.png', 'captcha_images\\232C.png', 'captcha_images\\235P.png', 'captcha_images\\235Y.png', 'captcha_images\\23

In [3]:
def get_CAPTCHA_label(captcha_image_file):
    """Get the CAPTCHA text from the file name."""
    filename = os.path.basename(captcha_image_file)
    label = filename.split(".")[0]
    return label

In [4]:
import cv2
def read_CAPTCHA_image(captcha_image_file):
    """Read CAPTCHA image into numpy array using OpenCv."""
    return cv2.imread(captcha_image_file)

In [5]:
def lets_see_it(image):
    """Displays the image."""
    cv2.imshow("window name", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
def grayscale_CAPTCHA_image(captcha_image):
    "Grayscales CAPTCHA image."""
    return cv2.cvtColor(captcha_image, cv2.COLOR_BGR2GRAY)

In [7]:
def threshold_CAPTCHA_image(captcha_image_grayscaled):
    """Thresholds CAPTCHA image."""
    return cv2.threshold(
        captcha_image_grayscaled, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )[1]

In [8]:
import numpy as np
def dilate_characters(binary_image):
    """slightly expands the characters."""
    kernel = np.ones((2,2), np.uint8) 
    return cv2.dilate(binary_image, kernel,iterations = 1)

In [9]:
def find_CAPTCHA_contours(captcha_image_thresholded):
    """Compute the contours of characters in the CAPTCHA image."""
    return cv2.findContours(
        captcha_image_thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )[0]

In [10]:
def compute_bounding_rectangles(contours):
    """Computes the bounding rectangles of the contours."""
    return list(map(cv2.boundingRect,contours))

In [11]:
def show_bounding_rectangles(rectangles, image):
    """Shows the bounding rectangles of contours on the image."""
    for rect in rectangles:
        x,y,w,h = rect
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    lets_see_it(image)

In [12]:
def split_fat_rectangles(rectangles):
    """Splits fat rectangles into two rectangles."""
    letter_bounding_rectangles = []
    for rectangle in rectangles:
        (x, y, w, h) = rectangle
        if w / h > 1.25:
            half_width = int(w / 2)
            letter_bounding_rectangles.append((x, y, half_width, h))
            letter_bounding_rectangles.append((x + half_width, y, half_width, h))
        else:
            letter_bounding_rectangles.append(rectangle)
    return letter_bounding_rectangles

In [13]:
def get_character_images(rectangles, image):
    """Extracts the characters defined by bounding rectangles."""
    char_images = []
    for rect in rectangles:
            x, y, w, h = rect
            char_image = image[y - 1 : y + h + 1, x - 1 : x + w + 1]
            char_images.append(char_image)
    return char_images

In [16]:
def sort_bounding_rectangles(rects): 
    """Sorts bounding rectangles by x coordinate."""
    return(sorted(rects, key = lambda x: float(x[0]))) 

In [17]:
captcha_processing_output_folder = "extracted_character_images"
character_counts = {}
for captcha_image_file in captchas:
    captcha_label = get_CAPTCHA_label(captcha_image_file)
    captcha_image = read_CAPTCHA_image(captcha_image_file)
    captcha_image_grayscaled = grayscale_CAPTCHA_image(captcha_image)
    captcha_image_thresholded = threshold_CAPTCHA_image(captcha_image_grayscaled)
    captcha_image_dilated = dilate_characters(captcha_image_thresholded)
    captcha_image_contours = find_CAPTCHA_contours(captcha_image_dilated)
    character_bounding_rectangles = split_fat_rectangles(compute_bounding_rectangles(captcha_image_contours))
    character_bounding_rectangles = sort_bounding_rectangles(character_bounding_rectangles)
    character_images = get_character_images(character_bounding_rectangles, captcha_image)
    for char_image, current_char in zip(character_images, captcha_label):
        if (len(character_images) == 4):
            save_dir = os.path.join(captcha_processing_output_folder, current_char)
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            character_count = character_counts.get(current_char, 0)
            image_save_path = os.path.join(save_dir, str(character_count) + ".png")
            cv2.imwrite(image_save_path, char_image)
            character_counts[current_char] = character_count + 1